# Rozpoznanie czerniaka na obrazku za pomocą prostej sieci konwolucyjnej

Importujemy biblioteki

In [1]:
import os
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Ładujemy obrazy treningowe.

In [2]:
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)
training_set = datagen.flow_from_directory('../gallery',
                                           target_size=(128, 128),
                                           batch_size=32,
                                           class_mode='binary',
                                           subset='training')
validation_set = datagen.flow_from_directory('../gallery',
                                           target_size=(128, 128),
                                           batch_size=32,
                                           class_mode='binary',
                                           subset='validation')

Found 17732 images belonging to 2 classes.
Found 4432 images belonging to 2 classes.


Budujemy sieć konwolucyjną z 3 warstwami konwolucyjnymi (ang. convolutional layer) i 2 warstwami gęstymi (ang. dense layer).

In [3]:
cnn = Sequential([
    # pierwsza warstwa konwolucyjna
    Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=(128, 128, 3)),
    MaxPool2D(pool_size=2, strides=2),
    Dropout(0.2),

    # druga warstwa konwolucyjna
    Conv2D(filters=64, kernel_size=3, activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    Dropout(0.2),

    # trzecia warstwa konwolucyjna
    Conv2D(filters=64, kernel_size=3, activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    Dropout(0.2),

    # trasformacja macierzy do tablicy (2D -> 1D)
    Flatten(),
    
    # 2 warstwy gęste
    Dense(units=128, activation='relu'),
    Dropout(0.2),
    
    Dense(units=128, activation='relu'),
    Dropout(0.2),

    # warstwa wyjściowa
    Dense(units=1, activation='sigmoid')
])

Kompilujemy poprzednio zbudowany model.

In [4]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Tworzymy odwołanie (ang. callback) do Tensorboard, żeby zebrać metryki.

In [5]:
log_dir = '../logs/fit/' + datetime.datetime.now().strftime('simple---%Y%m%d-%H%M%S')
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

Tworzymy odwołanie (ang. callback), które będzie zapisywać najlepszy model w trakcie trenowania.

In [6]:
checkpoint_path = 'checkpoints/simple-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    verbose=1,
    save_weights_only=False,
    save_freq='epoch',
    mode='auto')

Tworzymy odwołanie (ang. callback), które zatrzyma trenowanie, jeśli nie będzie progresu co najmniej 10 epoch z rzędu.

In [7]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.01, patience=10, restore_best_weights=True)

Trenujemy sieć konwolucyjną.

In [8]:
cnn.fit(training_set, validation_data=validation_set, epochs=200, callbacks=[tensorboard_callback,
                                                                     checkpoint_callback,
                                                                     early_stop_callback])

Epoch 1/200
555/555 [==============================] - 1944s 3s/step - loss: 0.3473 - accuracy: 0.8834 - val_loss: 0.2476 - val_accuracy: 0.8865

Epoch 00001: saving model to checkpoints/simple-0001.ckpt
INFO:tensorflow:Assets written to: checkpoints/simple-0001.ckpt/assets
Epoch 2/200
555/555 [==============================] - 1915s 3s/step - loss: 0.2940 - accuracy: 0.8858 - val_loss: 0.2454 - val_accuracy: 0.8872

Epoch 00002: saving model to checkpoints/simple-0002.ckpt
INFO:tensorflow:Assets written to: checkpoints/simple-0002.ckpt/assets
Epoch 3/200
555/555 [==============================] - 1917s 3s/step - loss: 0.2823 - accuracy: 0.8870 - val_loss: 0.2336 - val_accuracy: 0.8858

Epoch 00003: saving model to checkpoints/simple-0003.ckpt
INFO:tensorflow:Assets written to: checkpoints/simple-0003.ckpt/assets
Epoch 4/200
555/555 [==============================] - 1916s 3s/step - loss: 0.2597 - accuracy: 0.8902 - val_loss: 0.2985 - val_accuracy: 0.8858

Epoch 00004: saving model to 